In [10]:
import taichi as ti
import numpy as np
import math
from sympy import inverse_mellin_transform
from pyevtk.hl import gridToVTK
import pandas as pd
import matplotlib.pyplot as plt
import os
import argparse
import random

plt.rcParams['font.family'] = 'DeJavu Serif' 
plt.rcParams["font.serif"] = "Times New Roman"
ti.init(arch=ti.cpu, flatten_if=True)

dim = 2
Q = 9 
real = ti.f32
i32_vec2d = ti.types.vector(2, ti.i32)
f32_vec2d = ti.types.vector(2, ti.f32)
scalar_real = lambda: ti.field(dtype=real)
scalar_int = lambda: ti.field(dtype=ti.i32)
vec = lambda: ti.Vector.field(dim, dtype=real)
vec_Q = lambda: ti.Vector.field(Q, dtype=real)

# Input paramters
ly = 200
lx = 200
x = np.linspace(0, lx)
y = np.linspace(0, ly)
X, Y = np.meshgrid(x, y, indexing="ij")
IniPerturbRate = 1
rho0 = 0.5
rhos = 0.35
carn_star = False
T_Tc = 0.7
G = -1.0
tau = 1.  # specify the relaxaton time (only for BGK operator)
inv_tau = 1/tau
A = tau

dp = scalar_real() # pressure at left
loss = scalar_real()

is_solid = scalar_int()

v = vec()

temp_v = vec()
target_v =  vec() # target velocity field
rho = scalar_real()

collide_f = vec_Q()
stream_f = vec_Q()

ti.root.dense(ti.ij, (lx, ly)).place(is_solid)
ti.root.dense(ti.ij, (lx, ly)).place(target_v)
ti.root.dense(ti.ij, (lx, ly)).place(rho,v,temp_v)
ti.root.dense(ti.ij, (lx, ly)).place(collide_f,stream_f)

ti.root.place(dp, loss)

ti.root.lazy_grad()

"""Definition of LBM parameters"""
half = (Q - 1) // 2
# LBM weights
w0 = 4.0 / 9.0
w1 = 1.0 / 36.0
w2 = 1.0 / 9.0
w_np = np.array([w0, w1, w2, w1, w2, w1, w2, w1, w2])

# x component of predefined velocity in Q directions
e_xy_list = [
    [0, 0],
    [-1, 1],
    [-1, 0],
    [-1, -1],
    [0,-1],
    [1, -1],
    [1, 0],
    [1, 1],
    [0, 1],
]

# reversed_e_xy_np stores the index of the opposite component to every component in e_xy_np
# For example, for [1,0,0], the opposite component is [-1,0,0] which has the index of 2 in e_xy
reversed_e_index = np.array([e_xy_list.index([-a for a in e]) for e in e_xy_list])

w = ti.field(ti.f32,shape=Q)
w.from_numpy(w_np)
e_xy = ti.Vector.field(n=2, dtype=ti.i32, shape=Q)
e_xy.from_numpy(np.array(e_xy_list))
ti.static(e_xy)
ti.static(w)

# Writing input model (here we make a grain sample
def place_sphere(x, y, R):
    xmin = x - R
    ymin = y - R
    xmax = x + R
    ymax = y + R

    for px in range(xmin, xmax + 1):
        for py in range(ymin, ymax + 1):
            dx = px - x
            dy = py - y
        
            dist2 = dx * dx + dy * dy 
            R2 = R * R
            if dist2 < R2:
                near_px = (
                    math.floor(px + 0.5)
                    if math.floor(px + 0.5)
                    else math.floor(px + 0.5) + lx
                )
                near_py = (
                    math.floor(py + 0.5)
                    if math.floor(py + 0.5)
                    else math.floor(py + 0.5) + ly
                )
                
                if near_px >= lx:
                    near_px -= lx
                if near_py >= ly:
                    near_py -= ly
            
                is_solid[near_px, near_py] = 1


def read_positions(position_filename,reduced_R = True):
    global solid_count
    i = 0
    reduced = 1.
    with open(position_filename) as f:
        Lines = f.readlines()
    for line in Lines:
        if reduced_R:
            reduced = random.random()/10+0.9
        i += 1
        k = float(line)
        k = int(k)
        if i == 1:
            x = k
        elif i == 2:
            y = k
        
        else:
            i = 0
            r = int(max(k * reduced,k-1))
            if (y <= ly-r):
                solid_count += 1
                place_sphere(x, y, r)

os.chdir('./')
position_file = './positions_n375_res200_2d_random'
solid_count = 0
read_positions(position_file,reduced_R=True)
print(solid_count)

for j in range(ly):
    is_solid[0,j] = 0
    is_solid[lx-1,j] = 0

ti.static(is_solid)

[Taichi] Starting on arch=x64
194


<ti.field>

In [23]:
@ti.func
def periodic_index(i):
    iout = i
    if i[0]<0:     iout[0] = lx-1
    if i[0]>lx-1:  iout[0] = 0
    if i[1]<0:     iout[1] = ly-1
    if i[1]>ly-1:  iout[1] = 0

    return iout

@ti.func
def velocity_vec(local_pos) -> f32_vec2d:
    velocity_vec = ti.Vector([0., 0.])    
    for i in ti.static(range(2)):
        for s in ti.static(range(Q)):
            velocity_vec[i] = velocity_vec[i] + (stream_f[local_pos][s]* e_xy[s][i])
            
        velocity_vec[i] = velocity_vec[i]/rho[local_pos]

    return velocity_vec

@ti.func
def feq_p(k,rho_local, u): # anti bounce-back pressue bound
    eu = e_xy[k].dot(u)
    uv = u.dot(u)
    feqout = 2*w[k]*rho_local*(1.0+4.5*eu*eu-1.5*uv)
    return feqout

@ti.kernel
def init_field():
    for x, y in ti.ndrange(lx, ly):
        rho[x,y] = rhos
        v[x,y] = ti.Vector([0., 0.])
        collide_f[x,y] = ti.Vector([0.,0.,0.,0.,0.,0.,0.,0.,0.])
        stream_f[x,y] = ti.Vector([0., 0.,0., 0.,0., 0.,0., 0.,0.])

        if is_solid[x,y] <= 0:
            rho[x,y] = rho0
            for q in ti.static(range(Q)):
                collide_f[x,y][q] = w[q] * rho[x,y]
                stream_f[x,y][q] = w[q] * rho[x,y]      

@ti.kernel
def collision():
    for I in ti.grouped(collide_f):
        if (I.x < lx and I.y<ly and is_solid[I] <= 0):            
            """BGK operator"""
            v[I] = velocity_vec(I)
            u_squ = v[I].dot(v[I])
            for s in ti.static(range(Q)):
                eu = e_xy[s].dot(v[I])
                collide_f[I][s] = collide_f[I][s] + (w[s] * rho[I] *(1.0 + 3.0 * eu \
                    + 4.5 * eu * eu - 1.5 * u_squ) -collide_f[I][s]) *inv_tau

@ti.kernel
def post_collsion():
    for I in ti.grouped(collide_f):
        if (I.x < lx and I.y<ly and is_solid[I] <= 0):
            collide_f[I] = stream_f[I]
            rho[I] = collide_f[I].sum()

@ti.kernel
def boundary_condition():   
    for I in ti.grouped(v):
        if (I.x < lx and I.y<ly and is_solid[I]<=0):
            for s in ti.static(range(Q)):
                neighbor_pos = periodic_index(I+e_xy[s])
                
                if (e_xy[s][0] == 1 and I.x==0):
                    stream_f[I][s] = feq_p(s, rho0, v[I])- collide_f[I][reversed_e_index[s]]
                
                if (e_xy[s][0] == 1 and I.x==0 and I.y <100):
                    stream_f[I][s] = feq_p(s, dp[None], v[I])- collide_f[I][reversed_e_index[s]]

                if (e_xy[s][0] == -1 and I.x==lx-1):
                    stream_f[I][s]= feq_p(s, rho0, v[I])- collide_f[I][reversed_e_index[s]]
    
    for I in ti.grouped(v):
        if (I.x < lx and I.y<ly and is_solid[I] <= 0): 
            rho[I] = stream_f[I].sum()
            collide_f[I] = stream_f[I] 

@ti.kernel
def streaming():
    for I in ti.grouped(collide_f):
        if (I.x < lx and I.y<ly and is_solid[I] <= 0):
            for s in ti.static(range(Q)):
                neighbor_pos = periodic_index(I+e_xy[s])
                if (is_solid[neighbor_pos]<=0):
                    stream_f[neighbor_pos][s] = collide_f[I][s]
                else:
                    stream_f[I][reversed_e_index[s]] = collide_f[I][s]  

def export_VTK(n):
        is_solid_3d = np.ascontiguousarray(is_solid.to_numpy()[0:lx,0:ly]).reshape(lx,ly,1)
        rho_3d = np.ascontiguousarray(rho.to_numpy()[0:lx,0:ly]).reshape(lx,ly,1)
        v_ = v.to_numpy()[0:lx,0:ly,:]
        v_3d = v_[0:lx,0:ly,np.newaxis,:]

        max_rho = 0
        max_vx = 0
        max_vy = 0
        for i in range(lx):
            for j in range(ly):
                if is_solid_3d[i,j,0]<=0:
                    if rho_3d[i,j,0]>max_rho:
                        max_rho=rho_3d[i,j,0]
                    if v_[i,j,0] > max_vx:
                        max_vx = v_[i,j,0]
                    if v_[i,j,1] > max_vy:
                        max_vy = v_[i,j,1]
        
        grid_x = np.linspace(0, lx, lx)
        grid_y = np.linspace(0, ly, ly)
        # X, Y, Z= np.meshgrid(x, y, np.zeros(1), indexing='ij')
        z = np.array([0.0])

        gridToVTK(
                "./LB_SingelPhase_"+str(n),
                grid_x,
                grid_y,
                z,
                pointData={ "Solid": is_solid_3d,
                            "rho": rho_3d,
                            "velocity": (np.ascontiguousarray(v_3d[:,:,:,0]),
                                         np.ascontiguousarray(v_3d[:,:,:,1]),
                                         np.ascontiguousarray(v_3d[:,:,:,0]),
                                            )
                            }
            )  

@ti.kernel
def update_vel():
    for I in ti.grouped(v): 
        v[I] = velocity_vec(I)

def run(max_step=1000,compute_loss=True):
    step = 0
    init_field()
    while step < max_step:  
        boundary_condition()
        collision() 
        streaming()
        step +=1 
        if step%50 == 0:
            export_VTK(step//50)
    
    if compute_loss:
        update_vel()
        for i in range(lx):
            for j in range(ly):
                target_v[i,j] = v[i,j]

@ti.kernel
def compute_loss():  
    for I in ti.grouped(rho): 
        if (I.x < lx and I.y<ly and is_solid[I]<=0 and I.x >180):
            for i in ti.static(range(2)):
                temp_v[I][i]=0.
                for s in ti.static(range(Q)):
                    temp_v[I][i] = temp_v[I][i] + (stream_f[I][s]* e_xy[s][i])
                temp_v[I][i] = temp_v[I][i]/rho[I] 
                
            loss[None]+= (temp_v[I].norm()-target_v[I].norm())**2
        

In [24]:
# set target velocity field 
loss[None] = 0.
dp[None] = 0.8
target_p = dp[None]
run(max_step=3000)
print(loss[None])
compute_loss()
print(loss[None])

0.0
0.0


In [19]:
"""Success"""
# initial value of dp
dp[None] = 0.61
learning_rate =  0.0005
loss.grad[None] = 1.
init_p = dp[None]
init_loss = 0

# Arrays of deriavtives 
dp_array = []
loss_array = []


for iteration in range(100):
    init_field()
    for step in range(3000):
        with ti.ad.Tape(loss):
            boundary_condition()
            collision() 
            streaming()
            
            compute_loss()
            
    if iteration == 0:
            init_loss = loss[None]

    dp_array.append(dp[None])
    loss_array.append(loss[None])    
    
    print("dloss/ddp = {}, dp = {}, loss = {} at no. {} iteriation.".format(dp.grad[None],dp[None],loss[None],iteration))
    dp[None] -= dp.grad[None] * learning_rate

KeyboardInterrupt: 

In [ ]:
def dPress(rho_value,carn_star=False):
    if carn_star:
        a = 1.0
        b = 4.0
        R = 1.0
        Tc = 0.0943
        T = T_Tc * Tc
        eta = b * rho_value / 4.0
        eta2 = eta * eta
        eta3 = eta2 * eta
        rho2 = rho_value * rho_value
        one_minus_eta = 1.0 - eta
        one_minus_eta3 = one_minus_eta * one_minus_eta * one_minus_eta

        return rho_value * R * T * (1 + eta + eta2 - eta3) / one_minus_eta3 - a * rho2
    else:
        cs2 = 1.0 / 3.0
        psi = 1.0 - np.exp(-rho_value)
        psi2 = psi * psi
        return cs2 * rho_value + cs2 * G / 2 * psi2


def plot_curves(
    x,
    y,
    xlabel=None,
    ylabel=None,
    xlim=None,
    ylim=None,
    labels=None,
    colors=None,
    label_loc=None,
    save_fig=False,
    yscale=None,
    line_style=None,
    marker_style=['',''],
):

    if not labels:
        labels = []
        for i in range(len(x)):
            labels.append(i + 1)

    if line_style is not None:
        if colors:
            for i in range(len(x)):
                plt.plot(
                    x[i],
                    y[i],
                    linestyle=line_style[i],
                    marker=marker_style[i],
                    markersize=3,
                    color=colors[i],
                )
        else:
            for i in range(len(x)):
                plt.plot(x[i], y[i], linestyle=line_style[i], marker="o", markersize=2)
    else:
        if colors:
            for i in range(len(x)):
                plt.plot(
                    x[i], y[i], linestyle="-", marker="o", markersize=2, color=colors[i]
                )
        else:
            for i in range(len(x)):
                plt.plot(x[i], y[i], linestyle="-", marker="o", markersize=2)

    plt.legend(labels, fontsize=10)

    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel(xlabel, fontsize=12)
    plt.grid(True, color="k", linestyle="--")

    if save_fig:
        plt.savefig("Poiseuille_flow_velocity_profile", bbox_inches="tight", dpi=110)

In [ ]:
target_loss = 0

final_index = min(range(len(loss_array)), key=lambda i: abs(loss_array[i]-target_loss))
init_p_diff = dPress(init_p,carn_star=False)- dPress(rho0,carn_star=False)

target_p_diff = dPress(target_p,carn_star=False)- dPress(rho0,carn_star=False)
pressure_diff = dPress(np.array(dp_array),carn_star=False)-dPress(rho0,carn_star=False)
print(init_p_diff,target_p_diff)

In [ ]:
plt.rcParams['font.family'] = 'DeJavu Serif' 
plt.rcParams["font.serif"] = "Times New Roman"


fig,ax1 = plt.subplots(figsize=(8,5),dpi=120)
# ax1.set_xlim(0.99, 1.61)
# ax1.set_ylim(400, 1200)


ax1.scatter(init_p_diff,init_loss,color='blue',label = 'initial values')

ax1.scatter(target_p_diff,target_loss,color='red',label = 'target values',marker='p', s=80)

ax1.plot(pressure_diff, loss_array, linestyle='--',marker='o', markersize=5, 
markerfacecolor='none',color='black',label = 'trajectory of differentiation path')
# ax1.set_xlim([0.0,0.09])
# ax1.set_ylim([0, 120])

plt.xlabel(r'pressure difference ($\rm mu·lu^{-1}t^{-2}$)', fontsize=10)
plt.ylabel('loss', fontsize=10)

left, bottom, width, height = [0.6, 0.57, 0.28, 0.28]
ax2 = fig.add_axes([left, bottom, width, height])

ax2.scatter(target_p_diff,target_loss,color='red',label = 'target values',marker='p', s=80)

ax2.plot(pressure_diff, loss_array, linestyle='--',marker='o', markersize=5, 
markerfacecolor='none',color='black',label = 'trajectory of differentiation path')
# ax2.set_xlim([0.02, 0.06])
# ax2.set_ylim([-0.0, 0.1])

ax1.legend(fontsize=9,loc=(0.05,0.05))
plt.xlabel(r'pressure difference ($\rm mu·lu^{-1}t^{-2}$)', fontsize=9)
plt.ylabel('loss', fontsize=9)
plt.show()    